In [1]:
import pandas as pd
import numpy as np
from anytree import Node, RenderTree, AsciiStyle, LevelGroupOrderIter, LevelOrderGroupIter, PreOrderIter
from intervaltree import Interval, IntervalTree
import multiprocessing
from scripts.assembly_converter import convert_assembly_hg19_to_hg38

ModuleNotFoundError: No module named 'scripts.assembly_converter'

In [ ]:
df = convert_assembly_hg19_to_hg38(df)

In [2]:
df = pd.read_csv("artifacts/liftover_grch38_final.bed", sep="\t", header = None)
df.columns = ['chr', 'start', 'end', 'grch_37', 'orientation']
df['grch37_start'] = df['grch_37'].apply(lambda x: x.split(':')[1].split('-')[1])
df['grch37_chr'] = df['grch_37'].apply(lambda x: x.split(':')[0])
df['chr'] = df['chr'].apply(lambda x: x.replace('chr', ''))
df['grch37_chr'] = df['grch37_chr'].apply(lambda x: x.replace('chr', ''))
df

,chr,start,end,grch_37,orientation,grch37_start,grch37_chr
0,16,289439,289439,chr16:339440-339439,1,339439,16
1,16,289608,289608,chr16:339609-339608,1,339608,16
2,16,297721,297721,chr16:347722-347721,1,347721,16
3,16,310070,310070,chr16:360071-360070,1,360070,16
4,16,346146,346146,chr16:396147-396146,1,396146,16
...,...,...,...,...,...,...,...
1149,6,53161598,53161598,chr6:53026397-53026396,1,53026396,6
1150,2,219212438,219212438,chr2:220077161-220077160,1,220077160,2
1151,16,31459888,31459888,chr16:31471210-31471209,1,31471209,16
1152,18,61917060,61917060,chr18:59584294-59584293,1,59584293,18


In [4]:
cosmic = pd.read_csv('../data/COSMIC_driver_genes.csv')
cosmic = cosmic[~cosmic['Genome Location'].str.contains(':-')]
cosmic['chr'] = cosmic['Genome Location'].str.split(':').map(lambda x: x[0])
cosmic['start'] = cosmic['Genome Location'].str.split(':').map(lambda x: x[1]).str.split('-').map(lambda x: int(x[0]))
cosmic['end'] = cosmic['Genome Location'].str.split(':').map(lambda x: x[1]).str.split('-').map(lambda x: int(x[1]))
cosmic

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),...,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,chr,start,end
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,11.23,yes,NaN,melanoma,...,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG...",10,50799421,50885675
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,12.10,yes,NaN,AML,...,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00...",10,26746593,26860935
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,34.12,yes,NaN,"CML, ALL, T-ALL",...,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007...",9,130713946,130885683
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,25.20,yes,NaN,AML,...,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"ABLL,ARG,CCDS30947.1,ENSG00000143322.19,NM_007...",1,179099327,179229601
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,37.30,yes,NaN,lipoma,...,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"CCDS2516.1,CMKOR1,CXCR7,ENSG00000144476.5,GPR1...",2,236569641,236582358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,ZNF429,zinc finger protein 429,353088.0,19:21505564-21538078,2,NaN,12.00,yes,NaN,GBM,...,NaN,NaN,Mis,NaN,NaN,NaN,"CCDS42537.1,ENSG00000197013.9,NM_001001415.2,N...",19,21505564,21538078
732,ZNF479,zinc finger protein 479,90827.0,7:57119614-57139864,2,NaN,11.20,yes,NaN,"lung cancer, bladder carcinoma, prostate carci...",...,NaN,NaN,Mis,NaN,NaN,NaN,"CCDS43590.1,ENSG00000185177.12,KR19,NM_033273....",7,57119614,57139864
733,ZNF521,zinc finger protein 521,25925.0,18:25061926-25352152,1,NaN,11.20,yes,NaN,ALL,...,Dom,"oncogene, fusion",T,PAX5,NaN,NaN,"CCDS32806.1,EHZF,ENSG00000198795.10,Evi3,NM_01...",18,25061926,25352152
734,ZNRF3,zinc and ring finger 3,84133.0,22:28883592-29057487,2,NaN,12.10,yes,NaN,"colorectal cancer, adrenocortical carcinoma, g...",...,NaN,TSG,"N, F, Mis",NaN,NaN,NaN,"BK747E2.3,CCDS56225.1,ENSG00000183579.15,FLJ22...",22,28883592,29057487


In [18]:
cgc_tree = IntervalTree()

for index, row in cosmic.iterrows():
    cgc_tree.add(Interval(row['start'], row['end'], (row['chr'], row['Gene Symbol'], '0kb')))

    cgc_tree.add(Interval(row['start']-2000, row['start'], (row['chr'], row['Gene Symbol'], '2kb upstream')))
    cgc_tree.add(Interval(row['start']-10000, row['start']-2000, (row['chr'], row['Gene Symbol'], '10kb upstream')))
    cgc_tree.add(Interval(row['start']-100000, row['start']-10000, (row['chr'], row['Gene Symbol'], '100kb upstream')))

    cgc_tree.add(Interval(row['end'], row['end']+2000, (row['chr'], row['Gene Symbol'], '2kb downstream')))
    cgc_tree.add(Interval(row['end']+2000, row['end']+10000, (row['chr'], row['Gene Symbol'], '10kb downstream')))
    cgc_tree.add(Interval(row['end']+10000, row['end']+100000, (row['chr'], row['Gene Symbol'], '100kb downstream')))

df_tree = []

for index, row in df.iterrows():
    df_tree.append(Node(name = row['chr'] + ':' + str(row['start']) + '-' + str(row['end']), chr = row['chr'], start = row['start'], end = row['end']))

df['interactions'] = 0
df['interaction_gene'] = np.nan
df['interaction_info'] = np.nan

for index in range(len(df_tree)):
    child_list = [node for node in LevelOrderGroupIter(df_tree[index])][0]
    for node in child_list:
        found_list = list(set(list(cgc_tree.overlap(node.start, node.end)) + list(cgc_tree.at(node.start)) + list(cgc_tree.at(node.end))))  # use its coordinates to find interactions in the IntervalTree
        listindex = 0
        while listindex < len(found_list) and found_list[listindex].data[0] != node.chr:
            listindex += 1
        # print(len(found_list), listindex, node.chr, node.start, node.end)
        # print(found_list)
        if listindex < len(found_list):
            children_left = True
            found_interaction = found_list[listindex].data
            Node(name = found_interaction[0] + ':' +  str(found_list[listindex].begin) + '-' +  str(found_list[listindex].end), chr = found_interaction[0], start = found_list[listindex].begin, end = found_list[listindex].end, checked = 0, parent = node)
            # print("Added child to:", index)
            df.at[index, 'interactions'] = len(df_tree[index].descendants)
            df.at[index, 'interaction_gene'] = found_interaction[1]
            df.at[index, 'interaction_info'] = found_interaction[2]
    # if index == 19:
    #         break

df['grch37_start'] = pd.to_numeric(df['grch37_start'])

In [26]:
df[df['interactions'] > 0].drop_duplicates(keep='first')

,chr,start,end,grch_37,orientation,grch37_start,grch37_chr,interactions,interaction_gene,interaction_info
0,16,289439,289439,chr16:339440-339439,1,339439,16,1,AXIN1,0kb
1,16,289608,289608,chr16:339609-339608,1,339608,16,1,AXIN1,0kb
2,16,297721,297721,chr16:347722-347721,1,347721,16,1,AXIN1,0kb
3,16,310070,310070,chr16:360071-360070,1,360070,16,1,AXIN1,0kb
4,16,346146,346146,chr16:396147-396146,1,396146,16,1,AXIN1,0kb
...,...,...,...,...,...,...,...,...,...,...
1101,16,2254001,2254001,chr16:2304003-2304002,1,2304002,16,1,TRAF7,100kb downstream
1119,1,156738411,156738411,chr1:156708204-156708203,1,156708203,1,1,PRCC,100kb upstream
1121,12,57125241,57125241,chr12:57519025-57519024,1,57519024,12,1,STAT6,100kb downstream
1133,10,102470972,102470972,chr10:104230730-104230729,1,104230729,10,1,SUFU,100kb upstream


In [31]:
df.drop_duplicates(keep='first', inplace=True)

In [32]:
df.to_pickle('artifacts/gencode_interactions.pickle', compression='infer', protocol=5, storage_options=None)

In [9]:
df['interaction_gene'].value_counts()

TP53      64
TERT      40
RNF43     18
FOXA1     11
RB1       11
          ..
PMS2       1
ESR1       1
KMT2C      1
ACVR2A     1
BTK        1
Name: interaction_gene, Length: 153, dtype: int64

In [27]:
df['interaction_info'].value_counts()

0kb                 301
2kb downstream       49
100kb upstream       48
100kb downstream     47
10kb downstream      10
10kb upstream         2
2kb upstream          1
Name: interaction_info, dtype: int64